In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from tensorflow import keras 
import tensorflow as tf 
import random
import cv2
import os
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras import datasets, layers, models 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import shuffle 

#ignore warning messages
import warnings
warnings.filterwarnings('ignore') 

sns.set()

In [5]:
dataset = []

data_path = "../input/gestures/gestures_new"

for folder in os.listdir(data_path):

    sub_path=data_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        dataset.append(img_arr)

In [6]:
X = np.array(dataset)
X = X/255.0
X.shape

In [7]:
X_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
X_set = X_datagen.flow_from_directory(data_path, target_size = (224, 224), batch_size = 32, class_mode = 'sparse')

In [9]:
y = X_set.classes

In [10]:
X_set.class_indices

In [11]:
gestures_class = X_set.class_indices

In [12]:
print(X.shape)
print(y.shape)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

In [14]:
vgg = VGG19(input_shape=(224,224,3), weights='imagenet', include_top=False)

#do not train the pre-trained layers of VGG-19
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)

#adding output layer
prediction = Dense(8, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

In [15]:
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [16]:
#Early stopping to avoid overfitting of model
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [17]:
# fit the model
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test), callbacks=[early_stop], batch_size=32, shuffle=True)

In [18]:
history.history.keys()

In [19]:
model.evaluate(X_test,y_test,batch_size=32)

In [20]:
#predict
y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred,axis=1)

#get classification report
print(classification_report(y_pred,y_test))

#get confusion matrix

print(confusion_matrix(y_pred,y_test))

In [29]:
print(y_pred)
print(y_test)

In [36]:
fig, axes = plt.subplots(3,3, figsize=(10,10))
axes = axes.flatten()
for i,ax in enumerate(axes):
    g = random.randint(0, len(X_test))
    img = np.reshape(X_test[g], (224,224,3))
    ax.imshow(img)
    pred = y_test[g]
    p = str(pred)
    ax.set_title("Prediction: "+p)
    ax.grid()

In [33]:
model.save('hg_model_updated.h5')